In [3]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import itertools
import datetime
import time
import warnings
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import RepeatedKFold
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from mlxtend.regressor import StackingCVRegressor
import optuna
import seaborn as sns
random_state =42

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)



In [4]:
import pandas as pd

# CSV dosyalarını okuma
sub_df = pd.read_csv("/kaggle/input/gdz22-datathon-case-2-v3/2022gdz22-datathon-case-2/submission.csv")
test_df = pd.read_csv("/kaggle/input/gdz22-datathon-case-2-v3/2022gdz22-datathon-case-2/test.csv")
train_df = pd.read_csv('/kaggle/input/gdz22-datathon-case-2-v3/2022gdz22-datathon-case-2/train.csv')

# Verisetine etiket ekleme ve birleştirme
train_df['data'] = 'train'
test_df['data'] = 'test'
df = pd.concat([train_df, test_df])

# Nüfus verisini okuma ve düzenleme
nfs = pd.read_csv("../input/trkiye-ile-nfusu-ve-art-hz/turkiye_ilce_nufusu.csv")
nfs.columns = ["İLÇE", "Nüfus", "İlveilçemerkezleri", "Beldeveköyler", "Yıllıknüfusartışhızı"]
nfs["İLÇE"] = nfs["İLÇE"].str.upper().replace({
    "SEFERIHISAR": "SEFERİHİSAR", "DIKILI": "DİKİLİ", "TIRE": "TİRE", "AKHISAR": "AKHİSAR",
    "KIRAZ": "KİRAZ", "SALIHLI": "SALİHLİ", "ALIAĞA": "ALİAĞA", "ÖDEMIŞ": "ÖDEMİŞ",
    "ÇIĞLI": "ÇİĞLİ", "GAZIEMIR": "GAZİEMİR", "DEMIRCI": "DEMİRCİ", "ALAŞEHIR": "ALAŞEHİR",
    "SELENDI": "SELENDİ", "AHMETLI": "AHMETLİ"
})
nfs["Nüfus"] = nfs["Nüfus"].str.replace(" ", "").astype(int)

# Deneme2 verisini okuma ve düzenleme
deneme2 = pd.read_csv("../input/izmirmanisanufsrakm/deneme2.csv")
deneme2["genislik"] = deneme2["genislik"].str.replace(",", "").astype(float)

# Verileri birleştirme
df = df.merge(deneme2[["genislik", "İLÇE"]], on="İLÇE", how="left")
nfs.drop([114, 879], inplace=True)

df = df.merge(nfs[["İLÇE", "Nüfus"]], how="left")
df["oran"] = df["Nüfus"] / (df["genislik"] * 100)
df.drop(["Nüfus", "genislik"], axis=1, inplace=True)

# Tatil günlerini belirleme
df["holiday"] = 0
df["del_tarih"] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].str[:10]

holiday_dates = [
    '01.01.2021', '23.04.2021', '01.05.2021', '12.05.2021', '13.05.2021', '14.05.2021', '15.05.2021',
    '19.05.2021', '15.07.2021', '19.07.2021', '20.07.2021', '21.07.2021', '22.07.2021', '23.07.2021',
    '30.08.2021', '29.10.2021', '01.01.2022', '23.04.2022', '01.05.2022', '02.05.2022', '03.05.2022',
    '04.05.2022', '19.05.2022', '08.07.2022', '09.07.2022', '10.07.2022', '11.07.2022', '12.07.2022',
    '13.07.2022', '14.07.2022', '15.07.2022', '30.08.2022', '28.10.2022', '29.10.2022'
]

df.loc[df["del_tarih"].isin(holiday_dates), "holiday"] = 1
df.drop("del_tarih", axis=1, inplace=True)

# Tarih ve zaman işlemleri
df["BAŞLAMA_TARİHİ_VE_ZAMANI"] = pd.to_datetime(df["BAŞLAMA_TARİHİ_VE_ZAMANI"], format="%d.%m.%Y %H:%M:%S")
df["SONA_ERME_TARİHİ_VE_ZAMANI"] = pd.to_datetime(df["SONA_ERME_TARİHİ_VE_ZAMANI"], format="%d.%m.%Y %H:%M:%S")

# Zamanla ilgili ek değişkenler oluşturma
df['baslangic_month'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.month
df['sona_erme_month'] = df["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.month

df['baslangic_quarter'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.quarter
df['sona_erme_quarter'] = df["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.quarter

df['baslangic_day'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.day
df['sona_erme_day'] = df["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.day

df['baş_hour'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.hour
df['son_hour'] = df["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.hour

df['baslangic_dayofweek'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.dayofweek
df['sona_erme_dayofweek'] = df["SONA_ERME_TARİHİ_VE_ZAMANI"].dt.dayofweek

df['year'] = df["BAŞLAMA_TARİHİ_VE_ZAMANI"].dt.year


FileNotFoundError: [Errno 2] No such file or directory: '../input/izmirmanisanufsrakm/deneme2.csv'

In [5]:
assert df.shape[0] == 122143
assert df["OutageID"].value_counts().shape[0] == 122143
df.shape

(122143, 30)

In [4]:
# df["FeelsLikeC"] = np.sin((df["FeelsLikeC"]/8)+9.8)*20

In [5]:
def funk(x):
    if x <= 0.20:
        return "kısa"
    elif x > 0.20:
        return "uzun"
    else:
        return "normal"

df["SÜREYE_GÖRE"] = df["KESİNTİ_SÜRESİ"].apply(funk)

In [6]:
label = "cagri_count"
except_cols = ["KESİNTİ_NO", "KOD_NO", "ŞEBEKE_UNSURU_KODU", 'SONA_ERME_TARİHİ_VE_ZAMANI', 'OutageID', 'BİLDİRİME_GÖRE', 'data', 'KADEME']

categorical_cols = [
    "holiday", "İL", "İLÇE", "SÜREYE_GÖRE", "ŞEBEKE_UNSURU",
    "KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA", "KAYNAĞA_GÖRE", "SEBEBE_GÖRE",
    'baslangic_month', 'sona_erme_month', 'baslangic_quarter', 'sona_erme_quarter',
    'baslangic_day', 'sona_erme_day', 'baslangic_dayofweek', 'sona_erme_dayofweek',
    "baş_hour", "son_hour", "year"
]

# Eğitim verisetini ve etiketleri ayırma
X = df[df.data == 'train'].drop(labels=except_cols + [label], axis=1)
y = df[df.data == 'train'][label]

# Test verisetini ayırma
to_sub = df[df.data == 'test'].drop(labels=except_cols + [label], axis=1)

# Kategorik sütunları category tipine dönüştürme
for categorical_col in categorical_cols:
    X[categorical_col] = X[categorical_col].astype(str).astype("category")
    to_sub[categorical_col] = to_sub[categorical_col].astype(str).astype("category")


In [7]:
X.loc[X["KESİNTİ_SÜRESİ"] < -2, "KESİNTİ_SÜRESİ"] = 0
X.loc[X["KESİNTİ_SÜRESİ"] < 0, "KESİNTİ_SÜRESİ"] = X.loc[X["KESİNTİ_SÜRESİ"] < 0, "KESİNTİ_SÜRESİ"].abs()
X.loc[X["TOPLAM_KENTSEL_OG"] < 0, ["TOPLAM_KENTSEL_OG", "TOPLAM_KENTSEL_AG"]] = X.loc[X["TOPLAM_KENTSEL_OG"] < 0, ["TOPLAM_KENTSEL_OG", "TOPLAM_KENTSEL_AG"]].abs()
X.describe()

,KESİNTİ_SÜRESİ,KENTSEL_OG,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,KIRSAL_OG,KIRSAL_AG,TOPLAM_KENTSEL_OG,TOPLAM_KENTSEL_AG,TOPLAM_KENTALTI_OG,TOPLAM_KENTALTI_AG,TOPLAM_KIRSAL_OG,TOPLAM_KIRSAL_AG,oran
count,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000,105392.000000
mean,2.373879,7.799368,359.279860,1.234259,59.354069,2.370730,62.735701,7.551997,302.224957,0.966313,41.291327,1.768830,45.475548,12.667224
std,2.613654,36.745282,1690.032278,12.502370,584.891795,16.186787,382.329902,56.436233,1468.917225,18.799623,515.045242,20.171476,365.234550,29.960997
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023190
25%,0.931667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000564
50%,1.727222,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.098333,0.000000,0.000000,0.000000,0.000000,1.340373
75%,3.050278,0.000000,87.000000,0.000000,0.000000,0.000000,0.000000,0.000000,117.695000,0.000000,0.000000,0.000000,0.000000,3.464070
max,289.323333,1160.000000,70961.000000,735.000000,44781.000000,482.000000,11683.000000,6042.792222,129419.169441,3849.358333,74001.949997,2090.577778,31752.000000,140.227083


In [9]:
def kırsal_kent_kenta(df):
    X = df.copy()
    X["il_tipi"] = (
        (X["KENTSEL_OG"] + X["KENTSEL_AG"]).astype(str) + "-" +
        (X["KENTALTI_OG"] + X["KENTALTI_AG"]).astype(str) + "-" +
        (X["KIRSAL_OG"] + X["KIRSAL_AG"]).astype(str)
    )
    X["il_tipi"] = X["il_tipi"].apply(kentmaker)
    X["il_tipi"] = X["il_tipi"] + "-" + X["İL"].astype(str)
    X["il_tipi"] = X["il_tipi"].astype("category")
    return X

def kentmaker(x):
    x = list(map(int, x.split("-")))
    if x[0] >= x[1] and x[0] >= x[2]:
        return "kent"
    elif x[1] >= x[0] and x[1] >= x[2]:
        return "kentalt"
    else:
        return "kırsal"
    
X = kırsal_kent_kenta(X)
to_sub = kırsal_kent_kenta(to_sub)
categorical_cols.append("il_tipi")

In [10]:
def vakit_hafta_mevsim(y):
    month = y.month
    hour = y.hour
    day_of_week = y.dayofweek

    if month in [12, 1, 2, 3]:
        mevsim = "kış"
    elif month in [4, 5, 6]:
        mevsim = "ilkbahar"
    elif month in [7, 8, 9]:
        mevsim = "yaz"
    else:
        mevsim = "sonbahar"

    if hour in [0, 1, 2, 3, 4, 5]:
        vakit = "gece"
    elif hour in [6, 7, 8, 9, 10, 11, 12]:
        vakit = "sabah"
    elif hour in [13, 14, 15, 16, 17, 18]:
        vakit = "öglen"
    else:
        vakit = "aksam"
    
    if day_of_week in [5, 6]:
        gun = "hftsonu"
    else:
        gun = "hftiçi"

    return f"{vakit}-{gun}-{mevsim}"

X["vakit_hafta_mevsim"] = X["BAŞLAMA_TARİHİ_VE_ZAMANI"].apply(vakit_hafta_mevsim).astype("category")
to_sub["vakit_hafta_mevsim"] = to_sub["BAŞLAMA_TARİHİ_VE_ZAMANI"].apply(vakit_hafta_mevsim).astype("category")
caterogical_cols.append("vakit_hafta_mevsim")

In [11]:
def tplm_il_sebeb_sure(df):
    df_copy = df.copy()
    df_copy["new"] = df_copy["SEBEBE_GÖRE"].astype(str) + "-" + df_copy["İL"].astype(str) + "-" + df_copy["SÜREYE_GÖRE"].astype(str)
    df_copy["new"] = df_copy["new"].astype("category")
    return df_copy
######################################################################################################
X = tplm_il_sebeb_sure(X)
to_sub = tplm_il_sebeb_sure(to_sub)
caterogical_cols.append("new")

In [12]:
def en_son_skew(df):
    df_copy = df.copy()
    cols_to_transform = ['KENTSEL_OG', 'KENTSEL_AG', 'KENTALTI_OG', 'KENTALTI_AG', 
                         'KIRSAL_OG', 'KIRSAL_AG', 'TOPLAM_KENTALTI_OG', 
                         'TOPLAM_KENTALTI_AG', 'TOPLAM_KIRSAL_OG', 'TOPLAM_KIRSAL_AG']
    
    for col in cols_to_transform:
        if col in df_copy.columns:
            df_copy[col] = np.log1p(df_copy[col])
        else:
            print(f"Column '{col}' not found in DataFrame.")
    
    return df_copy
######################################################################################################
X = en_son_skew(X)
to_sub = en_son_skew(to_sub)


In [13]:
def outlier_thresholds(df, column, q1=0.01, q3=0.99):
    quartile1 = df[column].quantile(q1)
    quartile3 = df[column].quantile(q3)
    interquantile_range = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * interquantile_range
    lower_limit = quartile1 - 1.5 * interquantile_range
    return lower_limit, upper_limit

def replace_with_thresholds(df, column, q1=0.01, q3=0.99):
    lower_limit, upper_limit = outlier_thresholds(df, column, q1, q3)
    df[column] = df[column].apply(lambda x: lower_limit if x < lower_limit else upper_limit if x > upper_limit else x)
    return df

######################################################################################################
columns_to_check = ["KESİNTİ_SÜRESİ"]

for col in columns_to_check: 
    X = replace_with_thresholds(X, col, q1=0.01, q3=0.99)
    to_sub = replace_with_thresholds(to_sub, col, q1=0.01, q3=0.99)


In [14]:
def calculate_total_lines_and_duration(df):
    df["etkilenen_toplam_hat"] = (
        df["KENTSEL_AG"] +
        df["KENTALTI_AG"] +
        df["KIRSAL_AG"] +
        df["KENTSEL_OG"] +
        df["KENTALTI_OG"] +
        df["KIRSAL_OG"]
    )
    
    df["etkilenen_hatlar_toplam_süre"] = (
        df["TOPLAM_KENTSEL_OG"] +
        df["TOPLAM_KENTSEL_AG"] +
        df["TOPLAM_KENTALTI_OG"] +
        df["TOPLAM_KENTALTI_AG"] +
        df["TOPLAM_KIRSAL_OG"] +
        df["TOPLAM_KIRSAL_AG"]
    )
    
    df["oran"] *= df["etkilenen_toplam_hat"]
    
    return df

######################################################################################################
X = calculate_total_lines_and_duration(X)
to_sub = calculate_total_lines_and_duration(to_sub)

In [19]:
from sklearn.preprocessing  import RobustScaler
num_cols = X.select_dtypes(int)
train = X
test = to_sub

X_scaled = RobustScaler().fit_transform(train[num_cols])
X[num_cols] = pd.DataFrame(X_scaled, columns=train[num_cols].columns)

X_test_scaled = RobustScaler().fit_transform(test[num_cols])
to_sub[num_cols] = pd.DataFrame(X_test_scaled, columns=test[num_cols].columns)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=7 , shuffle=True, random_state=42)

import gc
gc.collect()
score_list = []
fold = 1
unseen_preds = []
importance = []
val_results=[]
for train_index,test_index in skf.split(X,y):
    X_train,X_val = X.iloc[train_index],X.iloc[test_index]
    y_train,y_val = y.iloc[train_index],y.iloc[test_index]
    
    print(X_train.shape,X_val.shape)
    cat = CatBoostRegressor(#**params,
                            iterations = 1000,loss_function='RMSE', eval_metric='RMSE',
                            allow_writing_files=False,
                            cat_features=caterogical_cols
                           )
    cat.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=500,verbose=200)
    
    sub_pred=cat.predict(to_sub)
    unseen_preds.append(sub_pred)
    val_results.append(cat.predict(X_val))
    score = mean_squared_error(y_val,cat.predict(X_val),squared=False)
    score_t = mean_squared_error(y_train,cat.predict(X_train),squared=False)
    print(f"RMSE FOLD-{fold}:{score}")
    print(f"RMSE Train FOLD-{fold}:{score_t}")
    print(f'Train Mean:{y_train.mean()}')
    print(f'Val Mean:{y_val.mean()}')
    print(f'Val Predicted Mean:{cat.predict(X_val).mean()}')
    print(f'Predicted Mean:{np.mean(sub_pred)}')
    score_list.append(score)
    importance.append(cat.get_feature_importance())
    fold+=1
print("CV Mean Score:",np.mean(score_list))

In [ ]:
preds=pd.DataFrame(np.mean(unseen_preds,axis=0).round(),columns=['cagri_count'])

for i in range(0,len(importance)):
    feature_importance = importance[i]
    sorted_idx = np.argsort(feature_importance)
    fig = plt.figure(figsize=(12, 6))
    plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
    plt.yticks(range(len(sorted_idx)), np.array(X.columns)[sorted_idx])
    plt.title('Feature Importance')
    
submission=pd.concat([df[df.data=='test']['KESİNTİ_NO'].reset_index(drop=True),preds.reset_index(drop=True)],axis=1)
submission.cagri_count=submission.cagri_count.astype(int)

In [ ]:
submission["cagri_count"].plot()

In [ ]:
submission.loc[submission["cagri_count"]<0,"cagri_count"] =0
submission.cagri_count=submission.cagri_count.astype(int)
submission.to_csv(f"submission-{np.mean(score_list)}.csv",index=False)
submission["cagri_count"].plot()

In [ ]:
assert False ,"Don't go there man!"

In [ ]:
lgbm_params = {'n_estimators': 100,'num_leaves': 14, 'max_depth': 8, 'learning_rate': 0.133462594683001, 'n_estimators': 132, 'min_child_weight': 0.40443820716736345}
enc = LabelEncoder()
for i in caterogical_cols:
    X[i] = enc.fit_transform(X[i])
    to_sub[i] = enc.transform(to_sub[i])

In [ ]:
random_state = 42
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=random_state)
def RMSE(estimator, X_train, Y_train, cv=rkf,n_jobs=4):
    cv_results = cross_val_score(estimator,X_train,Y_train,cv=cv,scoring="neg_mean_squared_error",n_jobs=n_jobs)
    return (np.sqrt(-cv_results)).mean()

In [ ]:
lgbm = LGBMRegressor(**lgbm_params,random_state=random_state).fit(X,y)
score = RMSE(lgbm, X, y)
print("½) sCoRe:",score )

In [ ]:
unseen_preds = lgbm.predict(to_sub)
preds=pd.DataFrame(unseen_preds.round(),columns=['cagri_count'])
submission=pd.concat([df[df.data=='test']['KESİNTİ_NO'].reset_index(drop=True),preds.reset_index(drop=True)],axis=1)
submission.cagri_count=submission.cagri_count.astype(int)
submission["cagri_count"].plot()

In [ ]:
submission["cagri_count"]= submission["cagri_count"].apply(np.abs)
submission.to_csv(f"submission-{score}+1.csv",index=False)
submission["cagri_count"].plot()

In [ ]:
feat_imp = pd.Series(lgbm.feature_importances_, index=X.columns)
feat_imp.nlargest(30).plot(kind='barh', figsize=(8,10))